In [10]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.6.3 (default, Oct  3 2017, 21:45:48) 
[GCC 7.2.0]
NLTK: 3.4.4
Scikit-learn: 0.21.3
Pandas: 0.25.0
Numpy: 1.17.0


In [11]:
# Carregando dados
import pandas as pd
import numpy as np

df = pd.read_csv('../../data/dataset_tratado.txt', header=None, encoding='utf-8', sep='\t')
df.fillna('x', inplace=True)
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 2 columns):
0    793 non-null bool
1    793 non-null object
dtypes: bool(1), object(1)
memory usage: 7.1+ KB
None
       0                                                  1
0  False  pretensa estadual deduz apel extrem esbarr ved...
1  False  embarg insist mesm razo recurs apresent oposic...
2  False  acorda ora recorr nao neg vigenc direit federa...
3  False  tribunal justic assent compreensa part det leg...
4  False  segund jurisprudenc superior tribunal justic a...


In [12]:
# verificação das classes dos acórdãos

classes = df[0]
print(classes.value_counts())

False    525
True     268
Name: 0, dtype: int64


In [13]:
from sklearn.preprocessing import LabelEncoder

# convertendo classes em binário
encoder = LabelEncoder()
binary = encoder.fit_transform(classes)

print(binary[:10])

[0 0 0 0 0 0 1 1 0 0]


In [14]:
# criação da lista de ementas
ementas = df[1]
print(ementas[:10])

0    pretensa estadual deduz apel extrem esbarr ved...
1    embarg insist mesm razo recurs apresent oposic...
2    acorda ora recorr nao neg vigenc direit federa...
3    tribunal justic assent compreensa part det leg...
4    segund jurisprudenc superior tribunal justic a...
5    sobrest recurs especial ate julgament recurs e...
6    inadmissivel exigenc recolh reu prisa requisit...
7    dentr limit leg vez caracteriz reincidenc agra...
8    sobr prescrica aca repetica indebit tributari ...
9    tratas agrav instrument interpost contr decisa...
Name: 1, dtype: object


In [15]:
# The following script uses the bag of words model to convert text documents into 
# corresponding numerical features:

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = vectorizer.fit_transform(ementas).toarray()

print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
#  The following script directly convert text documents into TFIDF feature values 
# (without first converting documents to bag of words features)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(ementas).toarray()

print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [17]:
# Like any other supervised machine learning problem, we need to divide data into 
# 20% test set and 80% training set

from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec


X_train, X_test, y_train, y_test = train_test_split(X, binary, test_size=0.2, random_state=0)

print(X_train)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs'),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 61.63522012578616
Decision Tree Accuracy: 86.79245283018868
Random Forest Accuracy: 86.16352201257862
Logistic Regression Accuracy: 74.84276729559748
SGD Classifier Accuracy: 79.24528301886792
Naive Bayes Accuracy: 71.69811320754717
SVM Linear Accuracy: 81.13207547169812


In [19]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs'),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


votingClassifier = VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1)
votingClassifier.fit(X_train, y_train)
y_pred = votingClassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 81.13207547169812


In [21]:
# relatório de classificação

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print((accuracy_score(y_test, y_pred)*100))

[[97  3]
 [27 32]]
              precision    recall  f1-score   support

           0       0.78      0.97      0.87       100
           1       0.91      0.54      0.68        59

    accuracy                           0.81       159
   macro avg       0.85      0.76      0.77       159
weighted avg       0.83      0.81      0.80       159

81.13207547169812
